In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pickle
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import defaultdict, Counter
from torch.cuda.amp import autocast, GradScaler
from torch.optim import *
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
os.environ["CUDA_VISIBLE_DEVICES"] = str(2)

# model

In [3]:
class WeightSeq(torch.nn.Module):
    def __init__(self, path=None):
        super(WeightSeq, self).__init__()

        self.item_num = 1410676
        self.seq_len = 0
        self.item_emb = torch.nn.Embedding(1410676, 128, padding_idx=0)
        self.emb_dropout = torch.nn.Dropout(p=0.0)
        self.max_ = None
        self.seq_len = 2
        self.mlp = nn.Sequential(
                torch.nn.Linear(128*4, 128*4),
                torch.nn.GELU(),
                torch.nn.Linear(128*4, 128)
            )
        self.reset_para()
        if path is not None:
            self.item_emb = torch.load(path)


    def seq_embed(self, input_ids, all_ids =None, index=None, seq_len=2):
        input_embed = self.emb_dropout(self.item_emb(input_ids)) 
        a = input_embed[:,:-4,:]
        mask = input_ids.clone()[:,:-4]
        mask[mask!=0] = 1
        a = (a*mask.unsqueeze(-1)).sum(dim=-2)
        mask = mask.sum(dim=-1).unsqueeze(-1)
        mask_t = mask.clone()
        mask_t[mask_t==0]=1
        a = a/mask_t
        mask[mask==0]=-1
        mask[mask>0]=0
        mask[mask==-1]=1
        add = self.emb_dropout(self.item_emb(torch.tensor([0]).cuda())).repeat(input_ids.shape[0],1)
        add = add *mask
        a = a+add
        
        b = input_embed[:,-4:-2,:]
        mask = input_ids.clone()[:,-4:-2]
        mask[mask!=0] = 1
        b = (b*mask.unsqueeze(-1)).sum(dim=-2)
        mask = mask.sum(dim=-1).unsqueeze(-1)
        mask_t = mask.clone()
        mask_t[mask_t==0]=1
        b = b/mask_t
        mask[mask==0]=-1
        mask[mask>0]=0
        mask[mask==-1]=1
        add = self.emb_dropout(self.item_emb(torch.tensor([0]).cuda())).repeat(input_ids.shape[0],1)
        add = add *mask
        b = b+add
        
        c = input_embed[:,-2,:]

        d = input_embed[:,-1,:]
        
        input_embed = self.mlp(torch.cat([a,b,c,d],dim=-1))
        input_embed = F.normalize(input_embed, p=2, dim=1)
        return input_embed
    
    def recall_embed(self, recall_ids):
        item_embed = self.emb_dropout(self.item_emb(recall_ids))
        item_embed = F.normalize(item_embed, p=2, dim=1)
        return item_embed
    
    def forward(self,
                input_ids,
                recall_items,
                all_ids = None,
                index = None,
                number = 0,
               ):
        input_embed = self.seq_embed(input_ids, all_ids, index, self.seq_len)
        item_embed = self.recall_embed(recall_items)
        
        logits = input_embed.mm(item_embed.T)
        
        return logits

    def reset_para(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        nn.init.normal_(self.item_emb.weight, std=0.01)

# load model

In [4]:
model = WeightSeq().cuda()
model.load_state_dict(torch.load('ckpt/task1-weight_embd2.pt'))
model.eval()

WeightSeq(
  (item_emb): Embedding(1410676, 128, padding_idx=0)
  (emb_dropout): Dropout(p=0.0, inplace=False)
  (mlp): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=512, out_features=128, bias=True)
  )
)

# only recall by mlp

In [5]:
for name in ['train']:
    if name == 'test':
        data = pd.read_pickle('../data/test_data_p2.dataset')
    elif name == 'train':
        data = pd.read_pickle('../data/train_data_005.dataset')[::10]
    elif name == 'valid':
        data = pd.read_pickle('../data/valid_data_005.dataset')

    session = data['session'].tolist()
    locale = data['locale'].tolist()
    locale2ids = pickle.load(open('../data/local2ids.pkl', 'rb'))

    recall_items = torch.tensor(locale2ids[locale[0]]).cuda()
    recall2 = []
    recall2_logits = []
    for i in tqdm(range(len(session))):
        if i !=0 and locale[i] != locale[i-1]:
            recall_items = torch.tensor(locale2ids[locale[i]]).cuda()
        row = session[i]
        if name != 'test':
            label = row[-1]
            row = row[:-1]
        row = [0,0,0,0,0] + list(dict.fromkeys(row[::-1]))[::-1]
        input_ids = torch.tensor([row]).cuda()
        with torch.no_grad():
            with autocast():
                pred = model(input_ids, recall_items)[0]
        idx = pred.topk(500).indices
        pred_items_ids = recall_items[idx.cpu().numpy()].tolist()
        logits = pred[idx].tolist()
        recall2.append(pred_items_ids)
        recall2_logits.append(logits)
    recall2_data = pd.DataFrame({'next_item_prediction':recall2,'nn_logits':recall2_logits})
    recall2_data.to_pickle('../data/recall2_'+name+'_200.dataset')

100%|██████████| 325635/325635 [2:24:29<00:00, 37.56it/s]  


# rule recall + mlp recall

In [6]:
for name in ['train']:
    if name == 'test':
        data = pd.read_pickle('../data/test_data_p2.dataset')
        recall_data = pd.read_pickle('../data/recall1_test_200_p2.dataset')
    elif name == 'train':
        data = pd.read_pickle('../data/train_data_005.dataset')[::10]
        recall_data = pd.read_pickle('../data/recall1_train_200_p2.dataset')
    elif name == 'valid':
        data = pd.read_pickle('../data/valid_data_005.dataset')
        recall_data = pd.read_pickle('../data/recall1_valid_200_p22.dataset')
    session = data['session'].tolist()
    locale = data['locale'].tolist()
    recall1 = recall_data['next_item_prediction']
    locale2ids = pickle.load(open('../data/local2ids.pkl', 'rb'))
    
    recall1_logits = []
    for i in tqdm(range(len(session))):
        recall_items = torch.tensor(recall1[i]).cuda()
        if recall_items.shape[0] == 0:
            recall1_logits.append([])
            continue
        row = session[i]
        if name != 'test':
            label = row[-1]
            row = row[:-1]
        row = [0,0,0,0,0] + list(dict.fromkeys(row[::-1]))[::-1]
        input_ids = torch.tensor([row]).cuda()
        with torch.no_grad():
            with autocast():
                pred = model(input_ids, recall_items)[0].tolist()
        recall1_logits.append(pred)
    
    recall2_data = pd.read_pickle('../data/recall2_'+name+'_200.dataset')
    recall2 = recall2_data['next_item_prediction'].tolist()
    recall2_logits = recall2_data['nn_logits'].tolist()
    real_recall = []
    real_logits = []
    rank = []
    for i in tqdm(range(len(recall2))):
        recall_temp = recall1[i][:200]
        logits_temp = recall1_logits[i][:200]
        rank_temp = list(range(len(logits_temp)))
        row = session[i]
        if name != 'test':
            row = row[:-1]
        for j in range(len(recall2[i])):
            if len(rank_temp)==250:
                break
            if recall2[i][j] in recall_temp:
                continue
            if recall2[i][j] in row:
                continue
            recall_temp.append(recall2[i][j])
            logits_temp.append(recall2_logits[i][j])
            if recall2[i][j] in recall1[i]:
                rank_temp.append(recall1[i].index(recall2[i][j]))
            else:
                rank_temp.append(-1)
        real_recall.append(recall_temp)
        real_logits.append(logits_temp)
        rank.append(rank_temp)
    df = pd.DataFrame({'next_item_prediction':real_recall,'nn_logits':real_logits,'rank':rank})
    df.to_pickle('../xgb-task1/recall_'+name+'_250_with_nn_p2.dataset')

100%|██████████| 325635/325635 [04:58<00:00, 1092.17it/s]


# only updata nn logits

In [9]:
model = WeightSeq().cuda()
model.load_state_dict(torch.load('ckpt/task1-weight_embd2.pt'))
model.eval()

WeightSeq(
  (item_emb): Embedding(1410676, 128, padding_idx=0)
  (emb_dropout): Dropout(p=0.0, inplace=False)
  (mlp): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=512, out_features=128, bias=True)
  )
)

In [17]:
name = ['train','test','valid'][2]
if name == 'test':
    data = pd.read_pickle('../data/test_data_p2.dataset')
    recall_data = pd.read_pickle('../xgb-task1/recall_'+name+'_250_with_nn_p2.dataset')
elif name == 'train':
    data = pd.read_pickle('../data/train_data_005.dataset')[::10]
    recall_data = pd.read_pickle('../xgb-task1/recall_'+name+'_250_with_nn_p2.dataset')
elif name == 'valid':
    data = pd.read_pickle('../data/valid_data_005.dataset')
    recall_data = pd.read_pickle('../xgb-task1/recall_'+name+'_250_with_nn_p2.dataset')
session = data['session'].tolist()
locale = data['locale'].tolist()
recall1 = recall_data['next_item_prediction']
locale2ids = pickle.load(open('../data/local2ids.pkl', 'rb'))

In [18]:
recall1_logits = []
for i in tqdm(range(len(session))):
    recall_items = torch.tensor(recall1[i]).cuda()
    if recall_items.shape[0] == 0:
        recall1_logits.append([])
        continue
    row = session[i]
    if name != 'test':
        label = row[-1]
        row = row[:-1]
    row = [0,0,0,0,0] + list(dict.fromkeys(row[::-1]))[::-1]
    input_ids = torch.tensor([row]).cuda()
    with torch.no_grad():
        with autocast():
            pred = model(input_ids, recall_items)[0].tolist()
    recall1_logits.append(pred)

100%|██████████| 16364/16364 [00:28<00:00, 573.95it/s]


In [19]:
df = pd.DataFrame({'nn_logits':recall1_logits})
df.to_pickle('../xgb-task1/logits_'+name+'_p2.dataset')
df

,nn_logits
0,"[0.77783203125, 0.01113128662109375, 0.1555175..."
1,"[0.6982421875, 0.64404296875, 0.54833984375, 0..."
2,"[0.859375, 0.84912109375, 0.67041015625, 0.809..."
3,"[0.7275390625, 0.71044921875, 0.81396484375, 0..."
4,"[0.1844482421875, 0.1629638671875, 0.158813476..."
...,...
16359,"[0.79248046875, 0.69091796875, 0.7470703125, 0..."
16360,"[0.8916015625, 0.865234375, 0.8583984375, 0.79..."
16361,"[0.85009765625, 0.783203125, 0.66455078125, 0...."
16362,"[0.82373046875, 0.84912109375, 0.7783203125, 0..."
